In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline, AutoTokenizer, TextClassificationPipeline, Pipeline
import torch

In [2]:
# tested in transformers==4.18.0 
import transformers
transformers.__version__


'4.29.2'

I want to define a new class as a subclass to bertforsequenceclassification that outputs the hidden states as well.

In [3]:
finbert = BertForSequenceClassification.from_pretrained("rifatozkurt/finbert-sentiment-v1",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("rifatozkurt/finbert-sentiment-v1")

In [4]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)

outputs = finbert(**inputs, output_hidden_states=True)
last_hidden_states = outputs.hidden_states[-1]

In [5]:
last_hidden_states.shape

torch.Size([1, 9, 768])

In [6]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
results = nlp(['growth is strong and we have plenty of liquidity.', 
               'tesla goes bankrupt',
              'formulation patents might protect Vasotec to a limited extent.'])

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [7]:
results

[{'label': 'LABEL_1', 'score': 0.9050427079200745},
 {'label': 'LABEL_2', 'score': 0.8791782259941101},
 {'label': 'LABEL_0', 'score': 0.969572126865387}]

In [8]:
import numpy as np

In [9]:
file1 = open("tesla-news-combined-dataset-duplicates-removed.csv", "r", encoding="utf-8")

list = []
lines = file1.readlines()
for line in lines:
    temp = line.rsplit(",", 2)
    list.append(temp)
newsarray = np.array(list)
headlines = newsarray[:,0].tolist()
dates = newsarray[:,1].tolist()
stocks = newsarray[:,2].tolist()
file1.close()

In [10]:
display(headlines)

['"Cowen Tech Media & Telecom Conference Begins Today, Presenters Include: AT&T, Match Group, Synaptics, Tesla, AMD, Check Point Technologies, GrubHub, Redfin, Carbon Black, Universal Display, Proofpoint, and Applied Optoelectronics"',
 '"Auto Stock Roundup: GM, Honda, Tesla And More"',
 '"Auto Stock Roundup: September China Sales Up, Autonomous & EV Move On, Tesla In Focus"',
 '"Benzinga\'s Bulls & Bears For The Past Week: Tesla And The Auto Parts Retailers"',
 '"Tesla CEO Musk Says Other Three Officers Should Be Charged In Floyd\'s Murder Case"',
 '"Apple Vs. Tesla: Morgan Stanley Breaks Down The Parallels"',
 '"Gene Munster Sees Apple And Tesla As Biggest Winners This Earnings Season"',
 '"Tesla, Wayfair And Other Profitable Short Trades Could Get A Short Covering Boost"',
 '"Jim Cramer Shares His Thoughts On Ambarella, Tesla Motors And AmerisourceBergen"',
 '"Midday Losers From September 15 - Tesla Motors Inc, Digital Ally, Inc. And More"',
 '"#PreMarket Primer: Wednesday, February

In [11]:
teslaresults = nlp(headlines)

In [12]:
teslaresults[0:5]

[{'label': 'LABEL_0', 'score': 0.9176052808761597},
 {'label': 'LABEL_1', 'score': 0.8789694905281067},
 {'label': 'LABEL_1', 'score': 0.9912698268890381},
 {'label': 'LABEL_1', 'score': 0.7483278512954712},
 {'label': 'LABEL_0', 'score': 0.5824039578437805}]

In [13]:
file2 = open("language-model-outputs.csv", "w", encoding="utf-8")

index = 0
for headline in headlines:
    file2.write(str(headline) + "," + str(dates[index]) + "," + str(stocks[index].strip()) + "," + str(teslaresults[index]["label"]) + "," + str(teslaresults[index]["score"]) + "\n")
    index += 1
file2.close()